## Аналіз A/B-тестів

Ви - аналітик даних в ІТ-компанії і до вас надійшла задача проаналізувати дані A/B тесту в популярній [грі Cookie Cats](https://www.facebook.com/cookiecatsgame). Це - гра-головоломка в стилі «з’єднай три», де гравець повинен з’єднати плитки одного кольору, щоб очистити дошку та виграти рівень. На дошці також зображені співаючі котики :)

Під час проходження гри гравці стикаються з воротами, які змушують їх чекати деякий час, перш ніж вони зможуть прогресувати або зробити покупку в додатку.

У цьому блоці завдань ми проаналізуємо результати A/B тесту, коли перші ворота в Cookie Cats було переміщено з рівня 30 на рівень 40. Зокрема, ми хочемо зрозуміти, як це вплинуло на утримання (retention) гравців. Тобто хочемо зрозуміти, чи переміщення воріт на 10 рівнів пізніше якимось чином вплинуло на те, що користувачі перестають грати в гру раніше чи пізніше з точки зору кількості їх днів з моменту встановлення гри.

Будемо працювати з даними з файлу `cookie_cats.csv`. Колонки в даних наступні:

- `userid` - унікальний номер, який ідентифікує кожного гравця.
- `version` - чи потрапив гравець в контрольну групу (gate_30 - ворота на 30 рівні) чи тестову групу (gate_40 - ворота на 40 рівні).
- `sum_gamerounds` - кількість ігрових раундів, зіграних гравцем протягом першого тижня після встановлення
- `retention_1` - чи через 1 день після встановлення гравець повернувся і почав грати?
- `retention_7` - чи через 7 днів після встановлення гравець повернувся і почав грати?

Коли гравець встановлював гру, його випадковим чином призначали до групи gate_30 або gate_40.

In [34]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.stats.api as sms
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from math import ceil
from statsmodels.stats.proportion import proportions_ztest, proportion_confint
%matplotlib inline

In [22]:
df = pd.read_csv("D:/Studing DATA/Model_python/data/statistical_hypothesis/statistical_hypothesis/cookie_cats.csv")

In [25]:
df.head()

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,0,0
1,337,gate_30,38,1,0
2,377,gate_40,165,1,0
3,483,gate_40,1,0,0
4,488,gate_40,179,1,1


In [26]:
df['retention_1'] = df['retention_1'].astype(int)
df['retention_7'] = df['retention_7'].astype(int)

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90189 entries, 0 to 90188
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userid          90189 non-null  int64 
 1   version         90189 non-null  object
 2   sum_gamerounds  90189 non-null  int64 
 3   retention_1     90189 non-null  int64 
 4   retention_7     90189 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 3.4+ MB


1. Для початку, уявімо, що ми тільки плануємо проведення зазначеного А/B-тесту і хочемо зрозуміти, дані про скількох користувачів нам треба зібрати, аби досягнути відчутного ефекту. Відчутним ефектом ми вважатимемо збільшення утримання на 1% після внесення зміни. Обчисліть, скільки користувачів сумарно нам треба аби досягнути такого ефекту, якщо продакт менеджер нам повідомив, що базове утримання є 19%.

In [28]:
effect_size = sms.proportion_effectsize(0.20, 0.19)

In [29]:
effect_size

np.float64(0.025241594409087353)

In [30]:
required_n = sms.NormalIndPower().solve_power(
    effect_size,
    power=0.8,
    alpha=0.05,
    ratio=1
    )                                                  # Розрахунок необхідного розміру вибірки

required_n = ceil(required_n)                          # Округлення до наступного цілого числа

print(required_n)

24638


2. Зчитайте дані АВ тесту у змінну `df` та виведіть середнє значення показника показник `retention_7` (утримання на 7 день) по версіям гри. Сформулюйте гіпотезу: яка версія дає краще утримання через 7 днів після встановлення гри?

In [32]:
df_gate30 = df[df['version'] == 'gate_30']
ret_7_mean_30 = df_gate30['retention_7'].mean()

df_gate40 = df[df['version'] == 'gate_40']
ret_7_mean_40 = df_gate40['retention_7'].mean()

print(f"Середнє retention_7 для gate_30 - {ret_7_mean_30*100:.2f}%")
print(f"Середнє retention_7 для gate_40 - {ret_7_mean_40*100:.2f}%")

Середнє retention_7 для gate_30 - 19.02%
Середнє retention_7 для gate_40 - 18.20%



**Гіпотези**   

H₀: μ = 19 % (всі користувачі утримуються )  
H₁: μ != 19 (після переміщення гейту на 40 рівень утримання змінилось)  
Тип тесту: двосторонній  

Видно наочно , що переміщення гейту вплинуло на утримання , середнє зменшилось тому не бачу сенсу перевіряти чи воно зросло,а от взагалі чи ця зміна значна треба перевірити.


3. Перевірте з допомогою пасуючого варіанту z-тесту, чи дає якась з версій гри кращий показник `retention_7` на рівні значущості 0.05. Обчисліть також довірчі інтервали для варіантів до переміщення воріт і після. Виведіть результат у форматі:

    ```
    z statistic: ...
    p-value: ...
    Довірчий інтервал 95% для групи control: [..., ...]
    Довірчий інтервал 95% для групи treatment: [..., ...]
    ```

    де замість `...` - обчислені значення.
    
    В якості висновку дайте відповідь на два питання:  

      1. Чи є статистична значущою різниця між поведінкою користувачів у різних версіях гри?   
      2. Чи перетинаються довірчі інтервали утримання користувачів з різних версій гри? Про що це каже?  


In [35]:
control_sample = df[df['version'] == 'gate_30'].sample(n=required_n, random_state=22)
treatment_sample = df[df['version'] == 'gate_40'].sample(n=required_n, random_state=22)

ab_test = pd.concat([control_sample, treatment_sample], axis=0)
ab_test.reset_index(drop=True, inplace=True)

In [36]:
ab_test.head()

,userid,version,sum_gamerounds,retention_1,retention_7
0,7540471,gate_30,45,1,0
1,3589138,gate_30,21,1,0
2,3177668,gate_30,14,1,0
3,2133884,gate_30,26,0,0
4,492763,gate_30,39,1,1


In [37]:
ab_test['version'].value_counts()

version
gate_30    24638
gate_40    24638
Name: count, dtype: int64

In [38]:
conversion_rates = ab_test.groupby('version')['retention_7']
conversion_rates = conversion_rates.agg(['mean', 'std', stats.sem])
conversion_rates.columns = ['conversion_rate', 'std_deviation', 'std_error']


conversion_rates.style.format('{:.5f}')

,conversion_rate,std_deviation,std_error
version,,,
gate_30,0.18914,0.39163,0.00249
gate_40,0.17798,0.38250,0.00244


In [39]:
control_results = ab_test[ab_test['version'] == 'gate_30']['retention_7']
treatment_results = ab_test[ab_test['version'] == 'gate_40']['retention_7']

In [40]:
n_con = control_results.count()
n_treat = treatment_results.count()
successes = [control_results.sum(), treatment_results.sum()]
nobs = [n_con, n_treat]

In [41]:
successes

[np.int64(4660), np.int64(4385)]

In [43]:
z_stat, pval = proportions_ztest(successes, nobs=nobs,alternative='two-sided')
(lower_con, lower_treat), (upper_con, upper_treat) = proportion_confint(successes, nobs=nobs, alpha=0.05)

print(f'z statistic: {z_stat:.2f}')
print(f'p-value: {pval:.3f}')
print(f'Довірчий інтервал 95% для групи control: [{lower_con:.3f}, {upper_con:.3f}]')
print(f'Довірчий інтервал 95% для групи treatment: [{lower_treat:.3f}, {upper_treat:.3f}]')

z statistic: 3.20
p-value: 0.001
Довірчий інтервал 95% для групи control: [0.184, 0.194]
Довірчий інтервал 95% для групи treatment: [0.173, 0.183]


   В якості висновку дайте відповідь на два питання:  

      1. Чи є статистична значущою різниця між поведінкою користувачів у різних версіях гри?  
      
      Так є! таким чином відхиляємо нульову гіпотезу. Ще раніше ми побачили, що переміщення знизило утримання, а тест довів, що це зниження дійсно шкідливе для утримання.
      
      2. Чи перетинаються довірчі інтервали утримання користувачів з різних версій гри? Про що це каже?  
      
      Візуально це підтверджує результати тесту: зміна є значущою.Тобто з високою ймовірністю можна сказати, що ретеншн для gate_40 справді нижчий, ніж для gate_30, а не просто випадкове коливання.

4. Виконайте тест Хі-квадрат на рівні значущості 5% аби визначити, чи є залежність між версією гри та утриманням гравця на 7ий день після реєстрації.

    - Напишіть, як для цього тесту будуть сформульовані гіпотези.
    - Проведіть обчислення, виведіть p-значення і напишіть висновок за результатами тесту.


**Гіпотези:**

* $H_0$: утримання однакова у всіх версіях.
* $H_a$: є різниця  між версіями.


In [51]:
pd.crosstab(ab_test['version'], ab_test['retention_7'])

retention_7,0,1
version,,
gate_30,19978,4660
gate_40,20253,4385


In [52]:
crosstab = pd.crosstab(ab_test['version'], ab_test['retention_7'])

In [53]:
chi2, p, dof, expected = stats.chi2_contingency(crosstab)

print(f"χ² = {chi2:.3f}")
print(f"p-value = {p:.5f}")
print(f"Ступені свободи = {dof}")
print("Очікувані частоти:\n", expected)

χ² = 10.166
p-value = 0.00143
Ступені свободи = 1
Очікувані частоти:
 [[20115.5  4522.5]
 [20115.5  4522.5]]


Інтерпретація:

p-value = 0.00143 < 0.05 - відхиляємо H₀.

Тобто різниця у retention між версіями статистично значуща.